In [43]:
#imports    
from pathlib import Path
from docx import Document
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langchain.embeddings import HuggingFaceEmbeddings # попробоавать другие эмбеддеры 
from langchain.vectorstores import FAISS #pinecone
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter #попробовать другие чанкеры
from langchain.schema import Document as LC_Document
from transformers import pipeline
from langchain.chains import RetrievalQA
# Заменить llm на другую по API

In [44]:

def load_docx_text(path: Path) -> str:
    """Читает .docx и возвращает весь текст из параграфов."""
    doc = Document(path)
    paragraphs = [p.text for p in doc.paragraphs if p.text.strip()]
    return "\n".join(paragraphs)

# Путь к папке data
data_dir = Path("data")

# Словарь: ключ — путь к файлу, значение — очищенный текст
texts = {}

# Проходим по всем .docx во вложенных папках
for file_path in data_dir.rglob("*.docx"):
    text = load_docx_text(file_path)
    texts[str(file_path)] = text
    print(f"Загружен: {file_path} (символов: {len(text)})")


Загружен: data/ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx (символов: 236099)
Загружен: data/СП 69.13330.2016 Подземные горные выработки. Актуализированная редакция СНиП 3.02.03-84 с Изменением No 1.docx (символов: 34447)
Загружен: data/Приказ-Ростехнадзора-от-08.12.2020-N-505-Об-утверждении-федеральных-норм-и-правил-в-области.docx (символов: 858180)
Загружен: data/СП 31-115-2006 Открытые плоскостные физкультурно-спортивные сооружения.docx (символов: 139622)
Загружен: data/СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx (символов: 152549)
Загружен: data/Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx (символов: 274294)
Загружен: data/СП 91.13330.2012 Подземные горные выработки. Актуализированная редакция СНиП II-94-80 с Изменением N 1.docx (символов: 119441)


In [45]:
for file_path in data_dir.rglob("*.docx"):
    doc = Document(file_path)
    raw = "\n".join(p.text for p in doc.paragraphs if p.text.strip())

    # Приводим к нижнему регистру
    cleaned = raw.lower()
    # Убираем всё, кроме букв (русских/латинских) и пробелов
    cleaned = re.sub(r'[^a-zа-яё\s]', ' ', cleaned)
    # Убираем цифры (если ещё остались)
    cleaned = re.sub(r'\d+', ' ', cleaned)
    # Собираем множественные пробелы и переносы в один пробел
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Обрезаем пробелы в начале и конце
    cleaned = cleaned.strip()

    texts[str(file_path)] = cleaned
    print(f"{file_path.name}: {len(raw)}→{len(cleaned)} символов после очистки")

ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx: 236099→221142 символов после очистки
СП 69.13330.2016 Подземные горные выработки. Актуализированная редакция СНиП 3.02.03-84 с Изменением No 1.docx: 34447→32485 символов после очистки
Приказ-Ростехнадзора-от-08.12.2020-N-505-Об-утверждении-федеральных-норм-и-правил-в-области.docx: 858180→825928 символов после очистки
СП 31-115-2006 Открытые плоскостные физкультурно-спортивные сооружения.docx: 139622→130727 символов после очистки
СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx: 152549→135571 символов после очистки
Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx: 274294→257827 символов после очистки
СП 91.13330.2012 Подземные горные выработки. Актуализированная редакция СНиП II-94-80 с Изменением N 1.docx: 119441→110980 символов после очистки


In [46]:
# Загружаем русский стоп-слова (один раз)
stop_ru = set(stopwords.words("russian"))

# Словарь для результатов
tokenized_texts = {}

for path, cleaned in texts.items():
    # 1) Разбиваем на токены
    raw_tokens = word_tokenize(cleaned, language="russian")
    # 2) Фильтруем: оставляем только слова (isalpha) и не стоп-слова
    filtered = [tok for tok in raw_tokens if tok.isalpha() and tok not in stop_ru]
    # 3) Сохраняем
    tokenized_texts[path] = filtered
    print(f"{path}: всего {len(raw_tokens)} токенов, после фильтрации {len(filtered)}")


data/ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx: всего 27743 токенов, после фильтрации 21523
data/СП 69.13330.2016 Подземные горные выработки. Актуализированная редакция СНиП 3.02.03-84 с Изменением No 1.docx: всего 4117 токенов, после фильтрации 3198
data/Приказ-Ростехнадзора-от-08.12.2020-N-505-Об-утверждении-федеральных-норм-и-правил-в-области.docx: всего 102898 токенов, после фильтрации 78487
data/СП 31-115-2006 Открытые плоскостные физкультурно-спортивные сооружения.docx: всего 17447 токенов, после фильтрации 13204
data/СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx: всего 17895 токенов, после фильтрации 13409
data/Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx: всего 31354 токенов, после фильтрации 24472
data/СП 91.13330.2012 Подземные горные выработки. Актуализированная редакция СНиП II-94-80 с Изменением N 1.docx: всего 14195 токенов

In [47]:
# 3) Подготовим список LC_Document с метаданными
lc_docs = []
for path, cleaned in texts.items():
    lc_docs.append(
        LC_Document(
            page_content=cleaned,
            metadata={"source": path}
        )
    )

# 4) Настраиваем чанкер
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # макс. символов в чанке
    chunk_overlap=50    # перекрытие между чанками
)

# 5) Разбиваем все документы на чанки
chunked_docs = text_splitter.split_documents(lc_docs)

# 6) Соберём результат по исходному пути
chunks_by_doc = {}
for doc in chunked_docs:
    src = doc.metadata["source"]
    chunks_by_doc.setdefault(src, []).append(doc.page_content)

# 7) Проверим
for src, chs in chunks_by_doc.items():
    print(f"{Path(src).name}: {len(chs)} чанков")

ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx: 492 чанков
СП 69.13330.2016 Подземные горные выработки. Актуализированная редакция СНиП 3.02.03-84 с Изменением No 1.docx: 73 чанков
Приказ-Ростехнадзора-от-08.12.2020-N-505-Об-утверждении-федеральных-норм-и-правил-в-области.docx: 1837 чанков
СП 31-115-2006 Открытые плоскостные физкультурно-спортивные сооружения.docx: 291 чанков
СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx: 302 чанков
Постановление_Правительства_РФ_от_16_02_2008_N_87_О_составе_разделов.docx: 574 чанков
СП 91.13330.2012 Подземные горные выработки. Актуализированная редакция СНиП II-94-80 с Изменением N 1.docx: 247 чанков


In [48]:
indexed_chunks = []

for doc_path, chunk_list in chunks_by_doc.items():
    for idx, chunk_text in enumerate(chunk_list, start=1):
        indexed_chunks.append({
            "doc_id": Path(doc_path).name,
            "chunk_id": idx,
            "text": chunk_text
        })

# Проверка: выведем первые 5
for item in indexed_chunks[:5]:
    print(item)


{'doc_id': 'ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx', 'chunk_id': 1, 'text': 'всп мо рф ведомственный свод правил нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры дата введения предисловие разработаны научно исследовательским центром центрального научно исследовательского института мо рф военным инженерно техническим университетом с использованием материалов научно производственного предприятия эра и войсковой части внесены военно научным комитетом службы расквартирования и обустройства мо рф утверждены и введены в действие'}
{'doc_id': 'ВСП 22-02-07МО РФ Нормы по проектированию устройству и эксплуатации молниезащиты объектов военной инфраструктуры.docx', 'chunk_id': 2, 'text': 'мо рф утверждены и введены в действие начальником службы расквартирования и обустройства министерства обороны российской федерации от сентября г в настоящих нормах реализованы требован

In [49]:
# 1. Инициализируем публичную SBERT-модель
hf_embed = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"}
)

In [50]:
# Пробуем получить эмбеддинг для примера
example = indexed_chunks[0]["text"]
vec = hf_embed.embed_documents([example])
print("Длина вектора:", len(vec[0]))

Длина вектора: 1024


In [51]:
# 1.1. Список текстов (чанков)
texts    = [chunk["text"]         for chunk in indexed_chunks]
# 1.2. Соответствующие метаданные
metadatas= [
    {"source": chunk["doc_id"], "chunk_id": chunk["chunk_id"]}
    for chunk in indexed_chunks
]

# 1.3. Запускаем векторизацию и создаём FAISS-индекс
faiss_store = FAISS.from_texts(
    texts,
    embedding=hf_embed,
    metadatas=metadatas
)

# 1.4.Сохраняем на диск, чтобы не пересчитывать каждый раз
faiss_store.save_local("faiss_index")

In [52]:
# 2.1. Создаём retriever — он будет возвращать топ-5 наиболее похожих чанков
retriever = faiss_store.as_retriever(search_kwargs={"k": 5})

# 2.2. Пробуем простой поиск
query = "в каких зонах по весу снежного покрова находятся Херсон и Мелитополь?"
relevant_docs = retriever.get_relevant_documents(query)

for doc in relevant_docs:
    print(doc.metadata, "→", doc.page_content[:200], "\n---\n")


{'source': 'СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx', 'chunk_id': 249} → высоты h м над уровнем моря начальная высота м относительно которой устанавливается высотный коэффициент принимаемая не менее м значения определяют по таблице е или по данным организаций по гидрометео 
---

{'source': 'СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx', 'chunk_id': 86} → земли для отдельных населенных пунктов российской федерации принимают в соответствии с приложением к для остальной территории российской федерации нормативное значение веса снегового покрова на м гори 
---

{'source': 'СП 20.13330.2016 Нагрузки и воздействия. Актуализированная редакция СНиП 2.01.07-85.docx', 'chunk_id': 85} → действием ветра или иных факторов принимаемый в соответствии с термический коэффициент принимаемый в соответствии с коэффициент формы учитывающий переход от веса снегового покрова земли к снеговой наг 
---

{'source': '

In [63]:
# 3) Загружаем seq2seq-модель RUT5 и оборачиваем в LangChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 3.1 Токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")
model     = AutoModelForCausalLM.from_pretrained( "ai-forever/rugpt3small_based_on_gpt2",
    use_safetensors=True,   # <-- загружаем через safetensors
    trust_remote_code=True  # если репо содержит кастомный код
)
# 3.2 Создаём HF-пайплайн для text2text
hf_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    device=-1,            # CPU
    max_length=150,
    do_sample=True,
    top_p=0.9,
    temperature=0.7
)
llm = HuggingFacePipeline(pipeline=hf_pipe)


Device set to use cpu


In [64]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Ты — эксперт по нормативным документам (СНиП, ВСП, постановления).
Используй только предоставленный контекст и дай короткий точный ответ.
Если ответа нет — напиши «Информация не найдена в документах».

Контекст:
{context}

Вопрос:
{question}

Ответ:
""".strip()
)

In [65]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [66]:
res = qa_chain({"query": "Что означает аббревиатура ТС?"})
print("Ответ:", res["result"].strip())

print("\nИсточники:")
for d in res["source_documents"]:
    print(f" • {d.metadata['source']} (чанк {d.metadata['chunk_id']})")


ValueError: Input length of input_ids is 515, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [60]:
answer = res["result"].replace("<extra_id_0>", "").strip()

In [57]:
# 1) Получаем релевантные чанки
docs = retriever.get_relevant_documents("Что означает аббревиатура ТС?")
# 2) Собираем context
context = "\n\n".join([d.page_content for d in docs])
# 3) Формируем полный промпт
full_prompt = prompt.format(context=context, question="Что означает аббревиатура ТС?")
print(full_prompt)   # посмотрите, что подставилось
# 4) Прогоняем через HF-pipeline напрямую
out = hf_pipe(full_prompt)[0]["generated_text"]
# 5) Чистый ответ без <extra_id_*>
print(out.replace("<extra_id_0>", "").strip())


Ты — эксперт по нормативным документам (СНиП, ВСП, постановления).
Используй только предоставленный контекст и дай короткий точный ответ.
Если ответа нет — напиши «Информация не найдена в документах».

Контекст:
в ежемесячно издаваемом информационном указателе национальные стандарты соответствующая информация уведомление и тексты размещаются также в информационной системе общего пользования на официальном сайте разработчика минрегион россии в сети интернет внесена опечатка сайт фау фцс по состоянию на введение актуализация настоящих норм проведена оао вними руководители темы д р техн наук проф д в яковлев д р техн наук проф м а розенбаум исполнители д р техн наук a m козел д р техн наук проф ю в громов

щупов и остальные величины могут быть считаны с основного поля дисплея после нажатием кнопки sel во время измерений прибор может подавать следующие звуковые сигналы непрерывный звуковой сигнал напряжение шума превышает в длинный звуковой сигнал после начала измерений когда напряжение шу